# Daily Stopwatch Data Science: KAThai Visualization

Note: this is a part of visualizing Khan Acadmy Thailand videos information. For more information, see https://khanacademythailand.org

## Stage 1: Ask a question

My objective is to create a visualization tool for Khan Academy Thailand videos.

There is no performance measure for the visualization yet.

## Stage 2: Set the environment up and get data

First, set up a directory for data and link it to this workplace. Download data into your choice of directory.

**Note:** in order to work properly, you need to set up a terminal with UTF-8 format. See more [here](http://stackoverflow.com/questions/10561923/unicodedecodeerror-ascii-codec-cant-decode-byte-0xef-in-position-1).

In [300]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [301]:
#Set up the environment
import pandas as pd                        #Pandas
import numpy as np                         #Numpy

#Reading directly from Google sheets is complicated. Let's do a simple version by going to a local CSV file. 
#For more information about Google Sheets API, see https://developers.google.com/sheets/api/quickstart/python
'''
from StringIO import StringIO              #Properly read goodle sheets got moved to io in python3.

import requests

#read Google sheets data
r = requests.get('https://docs.google.com/spreadsheets/d/1uJSAtyUTQX9pmbI8YlXg_3Z15vKy3Bu4-wvzmoYOep0&output=csv')
data = r.content

df = pd.read_csv(StringIO(data))#, index_col=0)
'''

# Set up data directory
DataDir = "C:/Users/Admin/Documents/data/"
filename = "ThaiKAVideoProcess.xlsx"

df = pd.pandas.read_excel(DataDir+filename,sheetname='AllAvailableThaiVideos')

In [302]:
df[:10]

,Domain,DomainThai,Subject,SubjectThai,Topic,TopicThai,Tutorial,TutorialThai,ThaiCurriculumMapping,IsComplete,...,KASiteNameThai,KA site link,dub (Y/N)?,sub (Y/N)?,Original YoutubeID,Subbed Youtube Link,Dubbed Youtube Link,Dubbed Youtube ID,[YoutubeEnglishName] [[YoutubeThaiName]],Note
0,Math,คณิตศาสตร์,Early math,คณิตปฐมวัย,Counting,การนับ,Counting small numbers,การนับจำนวนน้อยๆ,คณิตศาสตร์ - ประถมต้น...,1.0,...,การนับจำนวนน้อยๆ,https://www.khanacademy.org/math/early-math/cc...,NaN,Y,y2-uaPiyoxc,https://www.youtube.com/v/y2-uaPiyoxc&cc_load_...,NaN,NaN,NaN,NaN
1,Math,คณิตศาสตร์,Early math,คณิตปฐมวัย,Counting,การนับ,Counting small numbers,การนับจำนวนน้อยๆ,คณิตศาสตร์ - ประถมต้น...,1.0,...,การนับตามลำดับ,https://www.khanacademy.org/math/early-math/cc...,Y,Y,PEeUTQ0Gri8,https://www.youtube.com/v/PEeUTQ0Gri8&cc_load_...,https://www.youtube.com/v/h6OTaBtdZ20&cc_load_...,h6OTaBtdZ20,NaN,NaN
2,Math,คณิตศาสตร์,Early math,คณิตปฐมวัย,Counting,การนับ,Numbers 0 to 120,จำนวน 0 ถึง 120,คณิตศาสตร์ - ประถมต้น...,1.0,...,ตารางจำนวน,https://www.khanacademy.org/math/early-math/cc...,Y,Y,9XZypM2Z3Ro,https://www.youtube.com/v/9XZypM2Z3Ro&cc_load_...,https://www.youtube.com/v/WF1GsvP5_NU&cc_load_...,WF1GsvP5_NU,NaN,NaN
3,Math,คณิตศาสตร์,Early math,คณิตปฐมวัย,Counting,การนับ,Numbers 0 to 120,จำนวน 0 ถึง 120,คณิตศาสตร์ - ประถมต้น...,1.0,...,จำนวนที่หายไประหว่าง 0 ถึง 120...,https://www.khanacademy.org/math/early-math/cc...,Y,Y,1AqkBdCBm9o,https://www.youtube.com/v/1AqkBdCBm9o&cc_load_...,https://www.youtube.com/v/KppjuJsfn-U&cc_load_...,KppjuJsfn-U,NaN,NaN
4,Math,คณิตศาสตร์,Early math,คณิตปฐมวัย,Counting,การนับ,Counting objects,การนับวัตถุ,คณิตศาสตร์ - ประถมต้น...,1.0,...,การนับในภาพ,https://www.khanacademy.org/math/early-math/cc...,Y,Y,leDYnoNSvD4,https://www.youtube.com/v/leDYnoNSvD4&cc_load_...,https://www.youtube.com/v/nSWEDZFHYGs&cc_load_...,nSWEDZFHYGs,NaN,NaN
5,Math,คณิตศาสตร์,Early math,คณิตปฐมวัย,Counting,การนับ,Counting objects,การนับวัตถุ,คณิตศาสตร์ - ประถมต้น...,1.0,...,การนับวาฬ แกะ และดอกไม้...,https://www.khanacademy.org/math/early-math/cc...,NaN,Y,I9S5CvSqb5A,https://www.youtube.com/v/I9S5CvSqb5A&cc_load_...,NaN,NaN,NaN,NaN
6,Math,คณิตศาสตร์,Early math,คณิตปฐมวัย,Counting,การนับ,Counting objects,การนับวัตถุ,คณิตศาสตร์ - ประถมต้น...,1.0,...,การนับหมา หนู และคุกกี้...,https://www.khanacademy.org/math/early-math/cc...,NaN,Y,EUqhLxFccbM,https://www.youtube.com/v/EUqhLxFccbM&cc_load_...,NaN,NaN,NaN,NaN
7,Math,คณิตศาสตร์,Early math,คณิตปฐมวัย,Counting,การนับ,Comparing small numbers,การเปรียบเทียบจำนวนน้อยๆ...,คณิตศาสตร์ - ประถมต้น...,1.0,...,การเปรียบเทียบจำนวนสิ่งของ...,https://www.khanacademy.org/math/early-math/cc...,NaN,Y,__nkbr6DeTg,https://www.youtube.com/v/__nkbr6DeTg&cc_load_...,NaN,NaN,NaN,NaN
8,Math,คณิตศาสตร์,Early math,คณิตปฐมวัย,Counting,การนับ,Comparing small numbers,การเปรียบเทียบจำนวนน้อยๆ...,คณิตศาสตร์ - ประถมต้น...,1.0,...,การเปรียบเทียบจำนวนน้อยๆ บนเส้นจำนวน...,https://www.khanacademy.org/math/early-math/cc...,NaN,Y,tJrSILRXOUc,https://www.youtube.com/v/tJrSILRXOUc&cc_load_...,NaN,NaN,NaN,NaN
9,Math,คณิตศาสตร์,Early math,คณิตปฐมวัย,Counting,การนับ,Comparing small numbers,การเปรียบเทียบจำนวนน้อยๆ...,คณิตศาสตร์ - ประถมต้น...,1.0,...,นับตามประเภท,https://www.khanacademy.org/math/early-math/cc...,NaN,Y,UA975j_qsTQ,https://www.youtube.com/v/UA975j_qsTQ&cc_load_...,NaN,NaN,NaN,NaN


We will use an interactive visualization tool called Bokeh. See an [overview](http://nbviewer.jupyter.org/github/bokeh/bokeh-notebooks/blob/master/quickstart/quickstart.ipynb) and [how to install](http://bokeh.pydata.org/en/0.11.0/docs/installation.html).

In [303]:
from bokeh.models import ColumnDataSource, HoverTool, OpenURL, TapTool
from bokeh.plotting import figure, show
from bokeh.sampledata.periodic_table import elements
from bokeh.io import vplot

In [304]:
#make output appeared in notebook
from bokeh.plotting import output_notebook
output_notebook()

Loading BokehJS ...

## Stage 3: Explore the data

In [305]:
df = df[df.IsComplete==1]

In [306]:
df = df[df.Domain=='Math']

In [307]:
df.reset_index(inplace=True)

Here, we want to assign color based on the attributed called **ThaiCurriculumMapping**.

In [308]:
MappingList = list(df.ThaiCurriculumMapping.unique())
#l = pd.DataFrame(MappingList).sort()
#l
df['Color'] = "0"
for i in range(0,len(df)):
    for j in range(0,len(MappingList)):
        if df.ThaiCurriculumMapping[i] == MappingList[j]:
            df.loc[i,'Color'] = str(j)    

colormap = {
    "0"         : "#a6cee3",
    "1"         : "#1f78b4",
    "2"         : "#fdbf6f",
    "3"         : "#b2df8a",
    "4"         : "#33a02c",
    "alkali metal"         : "#a6cee3",
    "alkaline earth metal" : "#1f78b4",
    "halogen"              : "#fdbf6f",
    "metal"                : "#b2df8a",
    "metalloid"            : "#33a02c",
    "noble gas"            : "#bbbb88",
    "nonmetal"             : "#baa2a6",
    "transition metal"     : "#e08e79",
}

Now we want to assign group number to each row. According to the order, we know that the rows under the same topic/subject are next to each other. So we will assign number using loop. 

In [309]:
df['GroupNumber'] = 1
num = 1
for i in range(1,len(df)):
    if (df.Topic[i] == df.Topic[i-1])&(df.Subject[i] == df.Subject[i-1]):
        num = num + 1
    else:
        num = 1
    df.loc[i,'GroupNumber'] = num    
MaxGroupNumber = np.max(df['GroupNumber'])

group_range = [str(x) for x in range(1, MaxGroupNumber + 1)] #This is equivalent to Group number in Periodic Table (columns)

Next, let's detect the smaller category called Subject.

In [310]:
SubjectList = list(df.Subject.unique())
#l = pd.DataFrame(SubjectList).sort()
SubjectList

[u'Early math',
 u'Arithmetic',
 u'Pre-algebra',
 u'Basic geometry',
 u'Algebra basics',
 u'Algebra I',
 u'Geometry',
 u'Algebra II',
 u'Trigonometry',
 u'Probability and statistics',
 u'Precalculus',
 u'Differential calculus',
 u'Integral calculus',
 u'Grade 7 (NCERT) ']

In [311]:
SubplotList = [0]*len(SubjectList)

We build subplots for different subjects.

In [312]:
for i in range(0,len(SubjectList)):
#for i in range(13,14):
    df_small = df[df['Subject']==SubjectList[i]]
    #Here we will associate each row with each topic. This is equivalent to Roman Letter in Periodic Table (rows)
    TopicList = list(df_small.Topic.unique()) 
    source = ColumnDataSource(
        data=dict(
            GroupNumber= df_small['GroupNumber'],
            Topic = [str(x) for x in df_small['Topic']],
            Tutorial = df_small['Tutorial'],
            TutorialThai = df_small['TutorialThai'],
            KASiteName = df_small['KASiteName'],
            KASiteNameThai = df_small['KASiteNameThai'],
            KASiteLink = df_small['KA site link'],
            type_color= [colormap[x] for x in df_small['Color']],
        )
    )

    p = figure(title= SubjectList[i], tools="tap,hover",
               x_range=group_range, y_range=list(reversed(TopicList)), y_axis_label="Topic")
    p.plot_width = 1800
    p.plot_height = 24*(len(TopicList)+2)
    p.toolbar_location = None
    p.outline_line_color = None
    
    p.rect("GroupNumber", "Topic", 0.9, 0.9, source=source,
           fill_alpha=0.6, color="type_color")

    p.grid.grid_line_color = None
    
    #Hover to get information
    p.select_one(HoverTool).tooltips=[
                ("Tutorial","@Tutorial"),
                ("TutorialThai","@TutorialThai"),
                ("KASiteName","@KASiteName"),
                ("KASiteNameThai","@KASiteNameThai"),
                #("KASiteLink","@KASiteLink"),
            ]
    
    #Click to go to the website
    url = "@KASiteLink"
    taptool = p.select(type=TapTool)
    taptool.callback = OpenURL(url=url)
    
    SubplotList[i] = p

In [319]:
p=vplot(SubplotList[0],SubplotList[1],SubplotList[2],SubplotList[3],
    SubplotList[4],SubplotList[5],SubplotList[6],SubplotList[7],
    SubplotList[8],SubplotList[9],SubplotList[10],SubplotList[11],
    SubplotList[12],SubplotList[13])

In [320]:
show(p)

Here are some ideas to develop this further:

1. Create menu bar to switch subjects. See more [here](http://bokeh.pydata.org/en/0.11.0/docs/user_guide/interaction.html#userguide-interaction-actions-widget-callbacks)
2. Once clicked, give dialog box instead of just jump to URL directly (so can go to sub/dub/KA link).
3. Create channels to translators (video incomplete, please help). Use 'fill_alpha' to create such effect.
4. Do something to distinquish different tutorials under the same topic.

*Note to myself:* I spent 2 Pomodoros (or more) with a help from Tum to catch bugs in this program. One need to aware of characters ":" in the document. In addition, don't have NaN values. Spent too much time to catch bugs but it is working now. Let's do a better version next round. 